In [1]:
# Import required libraries
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim

# Define the transforms to be applied to the MNIST dataset
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5,), (0.5,))])

# Load the training set and apply the defined transforms
trainset = torchvision.datasets.MNIST(root='./data', train=True,
                                        download=True, transform=transform)

# Create a data loader for the training set
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64,
                                          shuffle=True, num_workers=2)

# Load the test set and apply the defined transforms
testset = torchvision.datasets.MNIST(root='./data', train=False,
                                       download=True, transform=transform)

# Create a data loader for the test set
testloader = torch.utils.data.DataLoader(testset, batch_size=64,
                                         shuffle=False, num_workers=2)

# Define the neural network architecture
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, 5)
        self.fc1 = nn.Linear(64 * 4 * 4, 1024)
        self.fc2 = nn.Linear(1024, 10)

    def forward(self, x):
        x = self.pool(nn.functional.relu(self.conv1(x)))
        x = self.pool(nn.functional.relu(self.conv2(x)))
        x = x.view(-1, 64 * 4 * 4)
        x = nn.functional.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Create an instance of the neural network
net = Net()

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

# Train the neural network
for epoch in range(5):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 200 == 199:    # print every 200 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 200))
            running_loss = 0.0

print('Finished Training')

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./data\MNIST\raw\train-images-idx3-ubyte.gz to ./data\MNIST\raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./data\MNIST\raw\train-labels-idx1-ubyte.gz to ./data\MNIST\raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./data\MNIST\raw\t10k-images-idx3-ubyte.gz to ./data\MNIST\raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./data\MNIST\raw\t10k-labels-idx1-ubyte.gz to ./data\MNIST\raw

[1,   200] loss: 0.308
[1,   400] loss: 0.092
[1,   600] loss: 0.070
[1,   800] loss: 0.060
[2,   200] loss: 0.046
[2,   400] loss: 0.036
[2,   600] loss: 0.038
[2,   800] loss: 0.040
[3,   200] loss: 0.026
[3,   400] loss: 0.028
[3,   600] loss: 0.027
[3,   800] loss: 0.026
[4,   200] loss: 0.019
[4,   400] loss: 0.016
[4,   600] loss: 0.028
[4,   800] loss: 0.022
[5,   200] loss: 0.014
[5,   400] loss: 0.016
[5,   600] loss: 0.015
[5,   800] loss: 0.017
Finished Training


In [2]:
# Evaluate the performance of the neural network on the test set
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 99 %


In [23]:
import cv2
import torch

# Load the image and resize it to 28x28 pixels
# Define a list to store the predicted classes
predicted_classes = []
# img = cv2.imread('./Nico_digits/IMG_0122.jpg')
for i in range(122, 132):
    # Load the image and preprocess it
    img = cv2.imread(f'./Nico_digits/IMG_0{i}.jpg')
    img = cv2.resize(img, (28, 28))

    # Convert the image to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Invert the image
    gray = 255 - gray

    # Normalize the pixel values to be between -1 and 1
    gray = (gray.astype('float32') - 127.5) / 127.5

    # Convert the image to a PyTorch tensor
    tensor = torch.from_numpy(gray).unsqueeze(0).unsqueeze(0)

    # Pass the tensor through the neural network
    output = net(tensor)

    # Find the index of the class with the highest probability
    _, predicted = torch.max(output.data, 1)

    # Add the predicted class to the list
    predicted_classes.append(predicted.item())

# Calculate the accuracy of the predictions
targets = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
accuracy = (sum([1 for i, j in zip(targets, predicted_classes) if i == j]) / len(targets)) * 100

print(f"The predicted classes are: {predicted_classes}")
print(f"The accuracy is: {accuracy}%")


The predicted classes are: [3, 1, 2, 3, 4, 3, 6, 7, 8, 9]
The accuracy is: 80.0%


The network classifies my '0' as a '3' and my '5' as a '3', so it is 80% accurate on this small testing set.